In [1]:
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch

In [2]:
#data feature
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import regionprops
from skimage.morphology import binary_erosion
from scipy.special import comb
import numpy.fft as fft


# get horitonzal edges from sobelY
def number_of_edgePixels(image):
    sobelY = cv2.Sobel(image, cv2.CV_64F,0,1,ksize=5)
    return np.count_nonzero(sobelY==255)

# std of mean of rows, get high value for black/white horisontal stripes.
def std_meanOfRows(image):
    mean_row = np.mean(image, axis=1)
    return np.std(mean_row, dtype=np.float32)

# Computes the Laplacian of the image and returns the mean value of all the pixels. 
# A higher value indicates more edges and details in the image
def mean_laplacian(image):
    return cv2.mean(cv2.Laplacian(image, cv2.CV_64F))[0]

# mean number of peak and valleys in each column, get high value for noisy pictures.
def mean_PeaksValleys(image):
    peaks_valleys = []
    for j in range(image.shape[1]):
        col = image[:, j]
        num_peaks, num_valleys = count_peaks_valleys(col)
        mean_peaks_valleys = (num_peaks + num_valleys) / 2
        peaks_valleys.append(mean_peaks_valleys)
    return np.mean(peaks_valleys, dtype=np.float32)
    
def count_peaks_valleys(arr_1d):
    num_peaks = 0
    num_valleys = 0
    for i in range(1, len(arr_1d) - 1):
        if arr_1d[i] > arr_1d[i-1] and arr_1d[i] > arr_1d[i+1]:
            num_peaks += 1
        elif arr_1d[i] < arr_1d[i-1] and arr_1d[i] < arr_1d[i+1]:
            num_valleys += 1
    return (num_peaks, num_valleys)

# numbers of edges from canny edge detection, use dfs to explore number of group of edge pixels
def num_edges_canny(image, L2Gradient, sobel_kernal_size):
    T_lower = 100
    T_upper = 200 
    edge = cv2.Canny(image, T_lower, T_upper, apertureSize=sobel_kernal_size, L2Gradient = L2Gradient)
    def dfs(r, c):
        if r < 0 or r >= np.size(image, 0) or c < 0 or c >= np.size(image, 1) or image[r][c] == 0:
            return 0
            
        image[r][c] = 0
            
        for i, j in zip((r - 1, r + 1, r, r), (c, c, c - 1, c + 1)):
            dfs(i, j)
            
        return 1
    
    return sum(dfs(i, j) for i in range(np.size(image, 0)) for j in range(np.size(image, 1)))

def lbp_histogram(image, radius, bins):
    # compute the LBP histogram of the image
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp, bins= bins, density=True)
    return np.ravel(hist)

# Haralick features 
# describe the texture of an image by computing statistics from the gray-level co-occurrence matrix (GLCM) of the image.
def haralick_contrast(image, distance=1, angle=0):
    glcm = graycomatrix(image, [distance], [angle], levels=256, symmetric=True, normed=True)
    return graycoprops(glcm, 'contrast')[0, 0]

#Hu moments are a set of shape features that are invariant to rotation, scale, and translation. 
# They can be computed from the moments of an image's contour
def hu_moments(image):
    moments = cv2.moments(image)
    hu_moments = cv2.HuMoments(moments)
    return np.ravel(hu_moments)

#set of shape features that are invariant to rotation, scale, and translation. 
# They can be computed from the radial and angular moments of an image's contour
def zernike_moments(image, radius=1, degree=8):
    eroded_image = binary_erosion(image, np.ones((2*radius+1, 2*radius+1)))
    props = regionprops(eroded_image.astype(np.uint8))
    moments = np.zeros(degree+1)
    for prop in props:
        r = np.sqrt(prop.area/np.pi)
        if r < radius:
            z = np.complex(prop.centroid[1], prop.centroid[0])
            for n in range(degree+1):
                for m in range(-n, n+1, 2):
                    if m < 0:
                        cmn = comb(n, int((n-m)/2))
                        fm = np.exp(np.complex(0, m*prop.orientation))
                        moments[n] += (r**n) * cmn

#GLCM-based texture features
def glcm_features(image):
    # Compute the gray-level co-occurrence matrix
    glcm = graycomatrix(image, [5], [0, np.pi/4, np.pi/2, 3*np.pi/4])
    
    # Compute some commonly used GLCM features
    contrast = graycoprops(glcm, 'contrast')
    dissimilarity = graycoprops(glcm, 'dissimilarity')
    homogeneity = graycoprops(glcm, 'homogeneity')
    energy = graycoprops(glcm, 'energy')
    correlation = graycoprops(glcm, 'correlation')
    
    # Concatenate the features into a single vector
    features = np.concatenate([contrast, dissimilarity, homogeneity, energy, correlation])
    
    return np.ravel(features)


# Fourier descriptors represent the shape of an object in terms of its frequency components
def fourier_shape_features(image):
    # Compute the Fourier Transform of the image
    f = fft.fft2(image)
    
    # Shift the zero frequency component to the center
    fshift = fft.fftshift(f)
    
    # Take the magnitude of the Fourier Transform
    magnitude_spectrum = np.log(np.abs(fshift))
    
    # Extract the Fourier coefficients for the first 5 frequencies in each dimension
    n = image.shape[0]
    m = image.shape[1]
    p = 5
    descriptors = np.zeros((p, p))
    for i in range(p):
        for j in range(p):
            descriptors[i, j] = np.abs(fshift[n//2 + i, m//2 + j])
    
    # Flatten the descriptors into a single vector
    features = descriptors.flatten()
    return np.ravel(features)


In [3]:
from sklearn import datasets
from sklearn.utils import Bunch


In [ ]:
# #example of using features to train model
# number_image = 2910
# feature = 9
# img=np.zeros(shape=(number_image,feature))
# for i in range(0,number_image): 
#     img[i][0] = number_of_edgePixels(npdata['data'][i])
#     img[i][1] = std_meanOfRows(npdata['data'][i])
#     img[i][2] = mean_PeaksValleys(npdata['data'][i])
#     img[i][3] = np.std(npdata['data'][i])
#     img[i][4] = mean_laplacian(npdata['data'][i])
#     img[i][5] = hu_moments(npdata['data'][i])[0]
#     img[i][6] = zernike_moments(npdata['data'][i])
#     img[i][7] = glcm_features(npdata['data'][i])[0][0]
#     img[i][8] = fourier_shape_features(npdata['data'][i])[0]
#     #print(fourier_shape_features(npdata['data'][i]).shape)
    
# print(img.shape)
# dataset = Bunch(data = img, target=npdata['label'])

In [5]:
def decision_tree_model(dataSet, testData_percentage, max_depth):
    img_train, img_test, label_train, label_test = train_test_split(dataSet.data, dataSet.target, test_size=testData_percentage, random_state=0)
    random_state = 0
    classifier = DecisionTreeClassifier(max_depth = 4, 
                             random_state = 0)
    
    #fitting classifier model
    classifier.fit(img_train, label_train)
    
    # classification_report outputs classification metrics
    # such as precision, recall and F1 score
    pred_result = classifier.predict(img_train)
    print('Classification Training Report: \n', classification_report(label_train, pred_result))
    
    # confusion_matrix outputs how many samples are correctly or incorrectly classified
    print('Train Confusion Matrix: \n', confusion_matrix(label_train, pred_result), "\n")

    # accuracy computes classification accuracy
    print('Train Accuracy: ', accuracy_score(label_train, pred_result), '\n')
    
    # testing with validate data
    validate_result = classifier.predict(img_test)
    print('Classification Testing Report: \n', classification_report(label_test, validate_result, zero_division=0))
    # `confusion_matrix` outputs how many samples are correctly or incorrectly classified
    print('Test Confusion Matrix: \n', confusion_matrix(label_test, validate_result), "\n")
    # `accuracy` computes classification accuracy
    print('Test Accuracy: ', accuracy_score(label_test, validate_result))

In [6]:
# load data from .npz file
npdata = np.load(f'/content/1776_datasetA_200_40.npz')
#npdata = np.load(f'/content/2910_datasetC_200x200.npz')
print(npdata['data'].shape)

number_image = 1776
radius = 8
bins = 8
total_feature_size = bins + 57

img_features=np.zeros(shape=(number_image, total_feature_size)) 
#print(fourier_shape_features(npdata['data'][0]).shape)
#img_features[i][20:] =  zernike_moments(npdata['data'][i])

for i in range(0,number_image):
     img_features[i][:bins] = lbp_histogram(npdata['data'][i], radius, bins)
     img_features[i][8] = number_of_edgePixels(npdata['data'][i])
     img_features[i][9] = std_meanOfRows(npdata['data'][i])
     img_features[i][10] =  mean_PeaksValleys(npdata['data'][i])
     img_features[i][11] = np.std(npdata['data'][i])
     img_features[i][12] = mean_laplacian(npdata['data'][i])
     img_features[i][13:20] = hu_moments(npdata['data'][i])[0:7]
     img_features[i][20:40] = glcm_features(npdata['data'][i])[0:20]
     img_features[i][40:65] = fourier_shape_features(npdata['data'][i])[0:25]

print(img_features.shape)

#add depth to image features
img_features = np.insert(img_features, bins, npdata['depth'], axis=1)
print(img_features.shape)

# make dataset in Bunch format 
dataset = Bunch(data = img_features, target=npdata['label'])


decision_tree_model(dataset, 0.2, 4)


(1776, 200, 200)
(1776, 65)
(1776, 66)
Classification Training Report: 
               precision    recall  f1-score   support

           0       0.95      0.83      0.89       837
           1       0.79      0.94      0.86       583

    accuracy                           0.87      1420
   macro avg       0.87      0.88      0.87      1420
weighted avg       0.89      0.87      0.88      1420

Train Confusion Matrix: 
 [[695 142]
 [ 36 547]] 

Train Accuracy:  0.8746478873239436 

Classification Testing Report: 
               precision    recall  f1-score   support

           0       0.92      0.83      0.87       206
           1       0.79      0.91      0.84       150

    accuracy                           0.86       356
   macro avg       0.86      0.87      0.86       356
weighted avg       0.87      0.86      0.86       356

Test Confusion Matrix: 
 [[170  36]
 [ 14 136]] 

Test Accuracy:  0.8595505617977528


In [10]:
from sklearn.model_selection import GridSearchCV

def decision_tree_model(dataSet, testData_percentage):
    img_train, img_test, label_train, label_test = train_test_split(dataSet.data, dataSet.target, test_size=testData_percentage, random_state=0)
    random_state = 0
    
    # Define the hyperparameters to tune and their possible values
    parameters = {'max_depth': [2, 4, 6, 8, 10, 12],
                  'criterion': ['gini', 'entropy']}
    
    # Create a Decision Tree Classifier object
    dt_classifier = DecisionTreeClassifier(random_state=random_state)
    
    # Create a Grid Search object
    grid_search = GridSearchCV(estimator=dt_classifier,
                               param_grid=parameters,
                               scoring='accuracy',
                               cv=5)
    
    # Fit the Grid Search object to the data
    grid_search.fit(img_train, label_train)
    
    # Get the best hyperparameters and their corresponding accuracy
    best_params = grid_search.best_params_
    best_accuracy = grid_search.best_score_
    print("Best Hyperparameters: ", grid_search.best_params_)
    print("Best Accuracy: ", grid_search.best_score_)
    
    # Train the Decision Tree Classifier with the best hyperparameters
    classifier = DecisionTreeClassifier(max_depth=best_params['max_depth'], 
                                         criterion=best_params['criterion'],
                                         random_state=random_state)
    
    # Fit the classifier model
    classifier.fit(img_train, label_train)
    
    # Classification report outputs classification metrics such as precision, recall and F1 score
    pred_result = classifier.predict(img_train)
    print('Classification Training Report: \n', classification_report(label_train, pred_result))
    
    # Confusion_matrix outputs how many samples are correctly or incorrectly classified
    print('Train Confusion Matrix: \n', confusion_matrix(label_train, pred_result), "\n")

    # Accuracy computes classification accuracy
    print('Train Accuracy: ', accuracy_score(label_train, pred_result), '\n')
    
    # Testing with validate data
    validate_result = classifier.predict(img_test)
    print('Classification Testing Report: \n', classification_report(label_test, validate_result, zero_division=0))
    
    # Confusion_matrix outputs how many samples are correctly or incorrectly classified
    print('Test Confusion Matrix: \n', confusion_matrix(label_test, validate_result), "\n")
    
    # Accuracy computes classification accuracy
    print('Test Accuracy: ', accuracy_score(label_test, validate_result))

    return classifier


In [11]:
decision_tree_model(dataset, 0.2)

Best Hyperparameters:  {'criterion': 'entropy', 'max_depth': 10}
Best Accuracy:  0.8338028169014085
Classification Training Report: 
               precision    recall  f1-score   support

           0       0.96      0.98      0.97       837
           1       0.96      0.94      0.95       583

    accuracy                           0.96      1420
   macro avg       0.96      0.96      0.96      1420
weighted avg       0.96      0.96      0.96      1420

Train Confusion Matrix: 
 [[817  20]
 [ 35 548]] 

Train Accuracy:  0.9612676056338029 

Classification Testing Report: 
               precision    recall  f1-score   support

           0       0.87      0.87      0.87       206
           1       0.83      0.82      0.82       150

    accuracy                           0.85       356
   macro avg       0.85      0.85      0.85       356
weighted avg       0.85      0.85      0.85       356

Test Confusion Matrix: 
 [[180  26]
 [ 27 123]] 

Test Accuracy:  0.851123595505618


DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)

In [12]:
def select_features(dataset, k):
    # Split the dataset into training and testing sets
    img_train, img_test, label_train, label_test = train_test_split(dataset.data, dataset.target, test_size=0.2, random_state=0)
    
    # Train a decision tree model using the full set of features
    classifier = DecisionTreeClassifier(max_depth = 10, random_state = 0)
    classifier.fit(img_train, label_train)
    
    # Get the feature importances
    feature_importances = classifier.feature_importances_
    
    # Sort the features by their importance
    sorted_idx = np.argsort(feature_importances)[::-1]
    
    # Select the top k features
    selected_features = sorted_idx[:k]
    
    # Train a new decision tree model using only the selected features
    classifier_selected = DecisionTreeClassifier(max_depth = 4, random_state = 0)
    classifier_selected.fit(img_train[:, selected_features], label_train)
    
    # Evaluate the model on the testing set
    pred_result = classifier_selected.predict(img_test[:, selected_features])
    print('Classification Testing Report: \n', classification_report(label_test, pred_result, zero_division=0))
    print('Test Confusion Matrix: \n', confusion_matrix(label_test, pred_result), "\n")
    print('Test Accuracy: ', accuracy_score(label_test, pred_result))
    
    # Return the selected features
    return selected_features


In [13]:
selected_features = select_features(dataset, 15)
print(selected_features)

Classification Testing Report: 
               precision    recall  f1-score   support

           0       0.93      0.82      0.87       206
           1       0.78      0.92      0.85       150

    accuracy                           0.86       356
   macro avg       0.86      0.87      0.86       356
weighted avg       0.87      0.86      0.86       356

Test Confusion Matrix: 
 [[168  38]
 [ 12 138]] 

Test Accuracy:  0.8595505617977528
[ 8 21 65 63 54  1 10 64 60 28 61 53 55 42 24]


In [16]:
def select_features(dataset, k):
    # Split the dataset into training and testing sets
    img_train, img_test, label_train, label_test = train_test_split(dataset.data, dataset.target, test_size=0.2, random_state=0)
    
    # Train a decision tree model using the full set of features
    classifier = DecisionTreeClassifier(max_depth=4, random_state=0)
    classifier.fit(img_train, label_train)
    
    # Get the feature importances
    feature_importances = classifier.feature_importances_
    
    # Sort the features by their importance
    sorted_idx = np.argsort(feature_importances)[::-1]
    
    # Select the top k features
    selected_features = sorted_idx[:k]
    
    # Train a new decision tree model using only the selected features
    classifier_selected = DecisionTreeClassifier(max_depth=4, random_state=0)
    classifier_selected.fit(img_train[:, selected_features], label_train)
    
    # Evaluate the model on the testing set
    pred_result = classifier_selected.predict(img_test[:, selected_features])
    print('Classification Testing Report: \n', classification_report(label_test, pred_result, zero_division=0))
    print('Test Confusion Matrix: \n', confusion_matrix(label_test, pred_result), "\n")
    print('Test Accuracy: ', accuracy_score(label_test, pred_result))
    
    # Return the selected features and the trained model
    return selected_features, classifier_selected, img_test


In [17]:
selected_features, classifier_selected, img_test = select_features(dataset, 15)
test_predictions = classifier_selected.predict(img_test[:, selected_features])


Classification Testing Report: 
               precision    recall  f1-score   support

           0       0.92      0.83      0.87       206
           1       0.79      0.91      0.84       150

    accuracy                           0.86       356
   macro avg       0.86      0.87      0.86       356
weighted avg       0.87      0.86      0.86       356

Test Confusion Matrix: 
 [[170  36]
 [ 14 136]] 

Test Accuracy:  0.8595505617977528
